In [42]:
# Use the testing mode for evaluation (the whole ready-to-go model), since we 
# are not contributing to the model itself
# If user would like to use this acceleration, select the menu option 
# "Runtime" -> "Change runtime type", select "Hardware Accelerator" -> "GPU" and
# click "SAVE"
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision.datasets import MNIST
from torchvision import transforms

from tqdm.autonotebook import tqdm
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import inspect
import time
from torch import nn, optim
import torch, os
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, CenterCrop, Grayscale
from torch.utils.data import DataLoader
from PIL import Image

from random import sample

# selected ResNet34 as the model
model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
classes = ["Apple Golden 1", "Apple Golden 2", "Apple Golden 3", "Apple Granny Smith", "Apple Pink Lady"]

def gettrainfiles(): 
    _files, classidx, _filefullpath = [], [], []
    for cls in range(len(classes)):
        classidx += [cls] * len(os.listdir(traindir + '\\' + classes[cls]))
        _files += os.listdir(traindir + '\\' + classes[cls])
        for f in os.listdir(traindir + '\\' + classes[cls]):
            _filefullpath.append(traindir + '\\' + classes[cls] + '\\' + f)
    return [_files, classidx, _filefullpath]
def gettestfiles(): 
    _files, classidx, _filefullpath = [], [], []
    for cls in range(len(classes)):
        classidx += [cls] * len(os.listdir(testdir + '\\' + classes[cls]))
        _files += os.listdir(testdir + '\\' + classes[cls])
        for f in os.listdir(testdir + '\\' + classes[cls]):
            _filefullpath.append(testdir + '\\' + classes[cls] + '\\' + f)
    return [_files, classidx, _filefullpath]

root_path = 'C:\\Users\\wangz\\Documents\\GitHub\\ECE561MachineVision\\FinalProject\\Q2\\fruits\\fruits-360_dataset\\fruits-360'
traindir = root_path + '\\Training'
testdir = root_path + '\\Test'

trainfiles = gettrainfiles()[0]
trainclasses = gettrainfiles()[1]
trainfilefullpaths = gettrainfiles()[2]
testfiles = gettestfiles()[0]
testclasses = gettestfiles()[1]
testfilefullpaths = gettestfiles()[2]

print("Training Data Size: " + str(len(trainfiles)))
print("Testing Data Size: " + str(len(testfiles)))

Using cache found in C:\Users\wangz/.cache\torch\hub\pytorch_vision_master


Training Data Size: 2413
Testing Data Size: 805


In [43]:
# Q2 (a) recognition experiment:
# here I have not yet started fine-tuning the network 
with open(root_path + '\\' + 'imagenet_classes.txt') as f:
    labels = [line.strip() for line in f.readlines()]

model.eval()

test_true, test_pred = [], []
sample_50_image_idx = [x for x in sample(range(len(gettestfiles()[0])),50)]
input_images = sample_50_image_paths = [Image.open(gettestfiles()[2][x]) for x in sample_50_image_idx]
test_true = [classes[testclasses[x]] for x in sample_50_image_idx]
for i in range(len(input_images)):
    preprocess = transforms.Compose([
                  transforms.CenterCrop(224),
                  transforms.ToTensor(),
                  transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                       std=[0.229, 0.224, 0.225]),])
    input_tensor = preprocess(input_images[i])
    input_batch = input_tensor.unsqueeze(0)
    # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
    _, index = torch.max(output, 1)
    # The output has unnormalized scores. To get probabilities, run a 
    # softmax on it (to normalize the scores to probability).
    percentage = torch.nn.functional.softmax(output, dim=1)[0] * 100
    pred_label_i = labels[index[0]]
    test_pred.append(pred_label_i)

print("Recognition Experiment Using ImageNet pre-trained model on the Fruit360 Data:")
print('\t', "True Fruit Class", '\t', "Predicted ImageNet Class")
print('\t', "################", '\t', "########################\n")

for i in range(len(sample_50_image_idx)):
    print('\t', test_true[i], '\t\t', test_pred[i])

Recognition Experiment Using ImageNet pre-trained model on the Fruit360 Data:
	 True Fruit Class 	 Predicted ImageNet Class
	 ################ 	 ########################

	 Apple Golden 3 		 Granny Smith
	 Apple Golden 2 		 Granny Smith
	 Apple Pink Lady 		 Granny Smith
	 Apple Pink Lady 		 Granny Smith
	 Apple Golden 2 		 Granny Smith
	 Apple Golden 3 		 Granny Smith
	 Apple Granny Smith 		 Granny Smith
	 Apple Pink Lady 		 orange
	 Apple Granny Smith 		 Granny Smith
	 Apple Pink Lady 		 Granny Smith
	 Apple Golden 2 		 Granny Smith
	 Apple Golden 2 		 Granny Smith
	 Apple Golden 2 		 Granny Smith
	 Apple Golden 1 		 lemon
	 Apple Granny Smith 		 Granny Smith
	 Apple Golden 2 		 Granny Smith
	 Apple Golden 2 		 Granny Smith
	 Apple Pink Lady 		 pomegranate
	 Apple Golden 3 		 Granny Smith
	 Apple Golden 1 		 lemon
	 Apple Pink Lady 		 orange
	 Apple Golden 1 		 custard apple
	 Apple Granny Smith 		 Granny Smith
	 Apple Pink Lady 		 Granny Smith
	 Apple Golden 2 		 Granny Smith
	 Apple

In [44]:
# Functional Utilities
class FiveClassFruit(Dataset):
    def __init__(self, filelist, classlist, fullfilepath, transform=None):
        self.filelist = filelist
        self.classlist = classlist
        self.fullfilepath = fullfilepath
        self.transform = transform
    
    def __len__(self):
        assert len(self.filelist) == len(self.classlist) == len(self.fullfilepath)
        return len(self.filelist)
    
    def __getitem__(self, idx):
        img_name = self.filelist[idx]            
        fullname = self.fullfilepath[idx]
        cls = self.classlist[idx]
        image = Image.open(fullname)
        if self.transform:
            image = self.transform(image)
        return [image, cls]

def get_data_loaders(train_batch_size=8, val_batch_size=2):    
    data_transform = Compose([Resize((224,224)),
                              CenterCrop(224),
                              ToTensor(), 
                              Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225])])

    train_loader = DataLoader(FiveClassFruit(trainfiles, 
                                             trainclasses,
                                             trainfilefullpaths, 
                                             transform=data_transform),
                              batch_size=train_batch_size, shuffle=True)

    val_loader = DataLoader(FiveClassFruit(testfiles, 
                                           testclasses,
                                           testfilefullpaths, 
                                           transform=data_transform),
                            batch_size=val_batch_size, shuffle=False)
    # use the train dataset for both train and validation (no labels on testset)
    return train_loader, val_loader

def calculate_metric(metric_fn, true_y, pred_y):
    # multi class problems need to have averaging method
    if "average" in inspect.getfullargspec(metric_fn).args:
        return metric_fn(true_y, pred_y, average="macro")
    else:
        return metric_fn(true_y, pred_y)
    
def print_scores(p, r, f1, a, batch_size):
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")

def train_and_val(model):
    batches = len(train_loader)
    val_batches = len(val_loader)

    # loop for every epoch (training + evaluation)
    for epoch in range(epochs):
        total_loss = 0

        # progress bar
        progress = tqdm(enumerate(train_loader), desc="Loss: ", total=batches)

        # ----------------- TRAINING  -------------------- 
        # set model to training
        model.train()
        for i, data in progress:
            X, y = data[0].to(device), data[1].to(device)

            # training step for single batch
            model.zero_grad()
            outputs = model(X)
            loss = loss_function(outputs, y)
            loss.backward()
            optimizer.step()

            # getting training quality data
            current_loss = loss.item()
            total_loss += current_loss

            # updating progress bar
            progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))

        # releasing unceseccary memory in GPU
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        # ----------------- VALIDATION  ----------------- 
        val_losses = 0
        precision, recall, f1, accuracy = [], [], [], []
        true_cls, pred_cls = [], []
        # set model to evaluating (testing)
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                X, y = data[0].to(device), data[1].to(device)
                # Get's the prediction (outputs) from the network
                outputs = model(X)
                val_losses += loss_function(outputs, y)

                predicted_classes = torch.max(outputs, 1)[1] # get class from network's prediction
                true_cls.extend(y.cpu())
                pred_cls.extend(predicted_classes.cpu())

                # calculate P/R/F1/A metrics for batch
                for acc, metric in zip((precision, recall, f1, accuracy), 
                                       (precision_score, recall_score, f1_score, accuracy_score)):
                    acc.append(calculate_metric(metric, 
                                                y.cpu(), predicted_classes.cpu()))
            print(confusion_matrix(true_cls, pred_cls, labels=[0,1,2,3,4]))
            print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
            print_scores(precision, recall, f1, accuracy, val_batches)
            losses.append(total_loss/batches) # for plotting learning curve
    print(f"Training time: {time.time()-start_ts}s")

In [45]:
# Use ResNet18 for fine-tuning fully connected layer to achieve the best results
model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
# Fine-tuning: First modify the model configurations
# set all weights to be fixed: not back-propogating gradients
# set the number of classes as 5 (required by the project)
# replace the last layer with our custmozied linear layer with 5 out features (5 classes)
class_num = len(classes)
channel_in = model.fc.in_features

model.fc = nn.Linear(in_features=channel_in, out_features=class_num)
for params in model.parameters():
    params.requires_grad = False

# set the fully connected layer to be trainable: require back-propogation of gradients
for params in model.fc.parameters():
    params.requires_grad = True

epochs = 3
train_loader, val_loader = get_data_loaders(64,16)# put your data loader here, play with batch size to satisfy cuda
loss_function = nn.CrossEntropyLoss() # your loss function, cross entropy works well for multi-class problems

# optimizer, try Adam this time to have a taste
# optimize the full connection layer only, magic numbers from online resource
optimizer = optim.Adam(model.fc.parameters(), lr=0.001, betas=(0.9, 0.999))

os.environ['CUDA_VISIBLE_DEVICES']='0'

start_ts = time.time()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = True

torch.cuda.empty_cache()
model.cuda()

losses = []

print("Pretrained ResNet18 with fine-tuning the fully-connected layer only:")
print("Batch Norms are all activated as pretraied parameters:")

train_and_val(model)

Using cache found in C:\Users\wangz/.cache\torch\hub\pytorch_vision_master


Pretrained ResNet18 with fine-tuning the fully-connected layer only:
Batch Norms are all activated as pretraied parameters:



[[164   0   0   0   0]
 [  0 164   0   0   0]
 [  0   0 161   0   0]
 [  0   0  15 149   0]
 [  0   0   0   0 152]]
Epoch 1/3, training loss: 0.7603741319555986, validation loss: 0.2977934181690216
	     precision: 0.9608
	        recall: 0.9516
	            F1: 0.9553
	      accuracy: 0.9816



[[164   0   0   0   0]
 [  0 164   0   0   0]
 [  0   0 161   0   0]
 [  0   0  17 147   0]
 [  0   0   0   0 152]]
Epoch 2/3, training loss: 0.19729715546495036, validation loss: 0.15635670721530914
	     precision: 0.9542
	        recall: 0.9439
	            F1: 0.9480
	      accuracy: 0.9792



[[164   0   0   0   0]
 [  0 164   0   0   0]
 [  0   0 161   0   0]
 [  0   0   8 156   0]
 [  0   0   0   0 152]]
Epoch 3/3, training loss: 0.1111740906183657, validation loss: 0.10285050421953201
	     precision: 0.9706
	        recall: 0.9657
	            F1: 0.9678
	      accuracy: 0.9902
Training time: 48.58374357223511s


In [46]:
# Use ResNet18 for testing Batch Normalization
# ResNet pretrained models already uses BatchNorm at each layer
# At this experiment, deactivate some BatchNorm children layers to see the performance difference
# Still need to modify the fc layer because we are dealing with a five-class classification problem
model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
# set the number of classes as 5 (required by the project)
# replace the last layer with our custmozied linear layer with 5 out features (5 classes)
class_num = len(classes)
channel_in = model.fc.in_features
model.fc = nn.Linear(in_features=channel_in, out_features=class_num)

# Deactivating the BatchNorms of the last layer
model.layer4[0].bn1 = nn.Identity()
model.layer4[0].bn2 = nn.Identity()
model.layer4[1].bn1 = nn.Identity()
model.layer4[1].bn2 = nn.Identity()


# Fine-tuning: First modify the model configurations
# set all weights to be fixed: not back-propogating gradients
for params in model.parameters():
    params.requires_grad = False

# set the fully connected layer to be trainable: require back-propogation of gradients
for params in model.fc.parameters():
    params.requires_grad = True



epochs = 10
train_loader, val_loader = get_data_loaders(64,16)# put your data loader here, play with batch size to satisfy cuda
loss_function = nn.CrossEntropyLoss() # your loss function, cross entropy works well for multi-class problems

# optimizer, try Adam this time to have a taste
# optimize the full connection layer only, magic numbers from online resource
optimizer = optim.Adam(model.fc.parameters(), lr=0.001, betas=(0.9, 0.999))

os.environ['CUDA_VISIBLE_DEVICES']='0'

start_ts = time.time()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = True

torch.cuda.empty_cache()
model.cuda()

losses = []

print("Pretrained ResNet18 with fine-tuning the fully-connected layer only:")
print("Last layer Batch Norm Children are all deactivated (replaced by Identitiy layer):\n")

train_and_val(model)

Using cache found in C:\Users\wangz/.cache\torch\hub\pytorch_vision_master


Pretrained ResNet18 with fine-tuning the fully-connected layer only:
Last layer Batch Norm Children are all deactivated (replaced by Identitiy layer):




[[164   0   0   0   0]
 [164   0   0   0   0]
 [161   0   0   0   0]
 [150   9   0   5   0]
 [140   0   0   0  12]]
Epoch 1/10, training loss: 1.5623458341548317, validation loss: 1.5269943475723267
	     precision: 0.2590
	        recall: 0.2167
	            F1: 0.2163
	      accuracy: 0.2218



[[164   0   0   0   0]
 [143  21   0   0   0]
 [102  26  32   1   0]
 [  3  14   0 147   0]
 [ 65   0   0   0  87]]
Epoch 2/10, training loss: 1.4755128527942456, validation loss: 1.447490930557251
	     precision: 0.6007
	        recall: 0.4508
	            F1: 0.4726
	      accuracy: 0.5662



[[164   0   0   0   0]
 [102  62   0   0   0]
 [ 34   1 119   7   0]
 [  0   0   0 164   0]
 [ 30   0   0   0 122]]
Epoch 3/10, training loss: 1.3949814281965558, validation loss: 1.3755168914794922
	     precision: 0.7824
	        recall: 0.6875
	            F1: 0.7068
	      accuracy: 0.7868


...


[[164   0   0   0   0]
 [  2 162   0   0   0]
 [  7  11 140   3   0]
 [  0   0   0 164   0]
 [  3   0   0   0 149]]
Epoch 6/10, training loss: 1.1926348711314954, validation loss: 1.185664176940918
	     precision: 0.8922
	        recall: 0.8795
	            F1: 0.8848
	      accuracy: 0.9681



[[164   0   0   0   0]
 [  2 162   0   0   0]
 [  6   2 153   0   0]
 [  0   0   0 164   0]
 [  4   0   0   0 148]]
Epoch 7/10, training loss: 1.1342423432751705, validation loss: 1.1340574026107788
	     precision: 0.9346
	        recall: 0.9259
	            F1: 0.9298
	      accuracy: 0.9828



[[164   0   0   0   0]
 [  1 163   0   0   0]
 [  5  10 146   0   0]
 [  0   0   0 164   0]
 [  0   0   0   0 152]]
Epoch 8/10, training loss: 1.0810902369649786, validation loss: 1.0807461738586426
	     precision: 0.9412
	        recall: 0.9326
	            F1: 0.9362
	      accuracy: 0.9804



[[164   0   0   0   0]
 [  0 164   0   0   0]
 [  3   6 150   2   0]
 [  0   0   0 164   0]
 [  0   0   0   0 152]]
Epoch 9/10, training loss: 1.0292262874151532, validation loss: 1.0355697870254517
	     precision: 0.9641
	        recall: 0.9594
	            F1: 0.9613
	      accuracy: 0.9865



[[164   0   0   0   0]
 [  0 164   0   0   0]
 [  4   5 152   0   0]
 [  0   0   0 164   0]
 [  0   0   0   0 152]]
Epoch 10/10, training loss: 0.9820238289080168, validation loss: 0.9894592761993408
	     precision: 0.9673
	        recall: 0.9630
	            F1: 0.9648
	      accuracy: 0.9890
Training time: 161.34204483032227s


In [47]:
# Use Alexnet for testing Dropout Regularization
# Alexnet already uses 0.5 in three dropout layers
# Modify the classfier[6] of Alexnet to fine tune
model = torch.hub.load('pytorch/vision', 'alexnet', pretrained=True)
model.classifier[6] = nn.Linear(4096, len(classes))

# set all weights to be fixed: not back-propogating gradients
for params in model.parameters():
    params.requires_grad = False

# set the classifier[6] layer to be trainable: require back-propogation of gradients
for params in model.classifier[6].parameters():
    params.requires_grad = True

epochs = 3
train_loader, val_loader = get_data_loaders(64,16)# put your data loader here, play with batch size to satisfy cuda
loss_function = nn.CrossEntropyLoss() # your loss function, cross entropy works well for multi-class problems

# construct the stochastic gradient descent opimizer 
# and filter-out those parameters requiring gradient inputs (previsou layers)
optimizer = torch.optim.SGD(params=filter(lambda p: p.requires_grad, model.parameters()), lr=0.1)

os.environ['CUDA_VISIBLE_DEVICES']='0'

start_ts = time.time()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = True

torch.cuda.empty_cache()
model.cuda()

losses = []

print("Pretrained Alexnet with fine-tuning the last linear layer only:")
print("Alexnet With Dropout Fully Activated:\n")
train_and_val(model)

Using cache found in C:\Users\wangz/.cache\torch\hub\pytorch_vision_master


Pretrained Alexnet with fine-tuning the last linear layer only:
Alexnet With Dropout Fully Activated:




[[164   0   0   0   0]
 [  0 163   1   0   0]
 [  0  15 146   0   0]
 [  0   0  23 141   0]
 [  0   0   0   0 152]]
Epoch 1/3, training loss: 53.99978729219813, validation loss: 0.4993555247783661
	     precision: 0.9216
	        recall: 0.8977
	            F1: 0.9055
	      accuracy: 0.9522



[[164   0   0   0   0]
 [  0 160   4   0   0]
 [  0   3 158   0   0]
 [  0   0  40 124   0]
 [  0   0   0   0 152]]
Epoch 2/3, training loss: 3.4772173696615747, validation loss: 0.7584784030914307
	     precision: 0.9314
	        recall: 0.9026
	            F1: 0.9104
	      accuracy: 0.9424



[[164   0   0   0   0]
 [  0 164   0   0   0]
 [  0  19 142   0   0]
 [  0   0   0 164   0]
 [  0   0   0   0 152]]
Epoch 3/3, training loss: 1.35492720572572, validation loss: 0.367942214012146
	     precision: 0.9706
	        recall: 0.9589
	            F1: 0.9613
	      accuracy: 0.9767
Training time: 26.92198085784912s


In [48]:
# Use Alexnet for testing Dropout Regularization
# Deactivating Dropout Layers by substituion of Identify layers
# Classifier[6] still needs to be modified because the number of classes has changed to 5
model = torch.hub.load('pytorch/vision', 'alexnet', pretrained=True)
model.classifier[6] = nn.Linear(4096, len(classes))

# deactivate the Dropout to see if there is difference or not
model.classifier[0] = nn.Identity()
model.classifier[3] = nn.Identity()

# set all weights to be fixed: not back-propogating gradients
for params in model.parameters():
    params.requires_grad = False

# set the classifier[6] layer to be trainable: require back-propogation of gradients
for params in model.classifier[6].parameters():
    params.requires_grad = True

epochs = 3
train_loader, val_loader = get_data_loaders(64,16)# put your data loader here, play with batch size to satisfy cuda
loss_function = nn.CrossEntropyLoss() # your loss function, cross entropy works well for multi-class problems

# construct the stochastic gradient descent opimizer 
# and filter-out those parameters requiring gradient inputs (previsou layers)
optimizer = torch.optim.SGD(params=filter(lambda p: p.requires_grad, model.parameters()), lr=0.1)

os.environ['CUDA_VISIBLE_DEVICES']='0'

start_ts = time.time()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = True

torch.cuda.empty_cache()
model.cuda()

losses = []

print("Pretrained Alexnet with fine-tuning the last linear layer only:")
print("Alexnet With Dropout Fully Deactivated (replaced by Identity layers):\n")
train_and_val(model)

Using cache found in C:\Users\wangz/.cache\torch\hub\pytorch_vision_master


Pretrained Alexnet with fine-tuning the last linear layer only:
Alexnet With Dropout Fully Deactivated (replaced by Identity layers):




[[164   0   0   0   0]
 [  0 160   4   0   0]
 [  0   0 161   0   0]
 [  0   0  30 134   0]
 [  0   0   0   0 152]]
Epoch 1/3, training loss: 44.93596063005297, validation loss: 0.4424959123134613
	     precision: 0.9412
	        recall: 0.9203
	            F1: 0.9278
	      accuracy: 0.9583



[[164   0   0   0   0]
 [  0 160   4   0   0]
 [  0   0 161   0   0]
 [  0   0  10 154   0]
 [  0   0   0   0 152]]
Epoch 2/3, training loss: 0.044896207357707774, validation loss: 0.08289152383804321
	     precision: 0.9510
	        recall: 0.9424
	            F1: 0.9461
	      accuracy: 0.9828



[[164   0   0   0   0]
 [  0 154  10   0   0]
 [  0   0 161   0   0]
 [  0   0  10 154   0]
 [  0   0   0   0 152]]
Epoch 3/3, training loss: 0.0036559014640117405, validation loss: 0.11428865790367126
	     precision: 0.9412
	        recall: 0.9289
	            F1: 0.9340
	      accuracy: 0.9755
Training time: 27.097511529922485s
